# TimeseriesGenerator intro

In [28]:
# univariate one step problem
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [29]:
# define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# define generator
n_input = 2
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=1)

# number of samples
print('Samples: %d' % len(generator))

# print each sample
for i in range(len(generator)):
    x, y = generator[i]
    print('%s => %s' % (x, y))

Samples: 8
[[1. 2.]] => [3.]
[[2. 3.]] => [4.]
[[3. 4.]] => [5.]
[[4. 5.]] => [6.]
[[5. 6.]] => [7.]
[[6. 7.]] => [8.]
[[7. 8.]] => [9.]
[[8. 9.]] => [10.]


# Multilayer Perceptron (MLP)

In [30]:
# define generator
n_input = 2
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=8)

In [31]:
# univariate one step problem with mlp
from numpy import array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# define generator
n_input = 2
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=8)

# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=200, verbose=0)

# make a one step prediction out of sample
x_input = array([9, 10]).reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[11.4589405]]


# LSTM

In [32]:
from tensorflow.keras.layers import LSTM

# define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

# reshape to [10, 1]
n_features = 1
series = series.reshape((len(series), n_features))

# define generator
n_input = 2
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=8)

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=0)

# make a one step prediction out of sample
x_input = array([9, 10]).reshape((1, n_input, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[10.716624]]


# Multivariate Time Series Example

In [33]:
from numpy import hstack

# define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])

# reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
print(dataset)

# define generator
n_input = 2
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=1)

# number of samples
print('Samples: %d' % len(generator))

# print each sample
for i in range(len(generator)):
    x, y = generator[i]
    print('%s => %s' % (x, y))

[[ 10  15]
 [ 20  25]
 [ 30  35]
 [ 40  45]
 [ 50  55]
 [ 60  65]
 [ 70  75]
 [ 80  85]
 [ 90  95]
 [100 105]]
Samples: 8
[[[10. 15.]
  [20. 25.]]] => [[30. 35.]]
[[[20. 25.]
  [30. 35.]]] => [[40. 45.]]
[[[30. 35.]
  [40. 45.]]] => [[50. 55.]]
[[[40. 45.]
  [50. 55.]]] => [[60. 65.]]
[[[50. 55.]
  [60. 65.]]] => [[70. 75.]]
[[[60. 65.]
  [70. 75.]]] => [[80. 85.]]
[[[70. 75.]
  [80. 85.]]] => [[90. 95.]]
[[[80. 85.]
  [90. 95.]]] => [[100. 105.]]


In [34]:
# define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])

# reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))

# define generator
n_features = dataset.shape[1]
n_input = 2
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=8)

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=0)

# make a one step prediction out of sample
x_input = array([[90, 95], [100, 105]]).reshape((1, n_input, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[110.45394 115.82713]]


# Multivariate Inputs and Dependent Series Example

In [35]:
from numpy import insert

# define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
out_seq = array([25, 45, 65, 85, 105, 125, 145, 165, 185, 205])

# reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
out_seq.shape

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))

# shift the target sample by one step
out_seq = insert(out_seq, 0, 0)
out_seq = out_seq[0:-1]

# define generator
n_input = 1
generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=1)

# print each sample
for i in range(len(generator)):
    x, y = generator[i]
    print('%s => %s' % (x, y))

[[[10. 15.]]] => [25.]
[[[20. 25.]]] => [45.]
[[[30. 35.]]] => [65.]
[[[40. 45.]]] => [85.]
[[[50. 55.]]] => [105.]
[[[60. 65.]]] => [125.]
[[[70. 75.]]] => [145.]
[[[80. 85.]]] => [165.]
[[[90. 95.]]] => [185.]


# Multi-step Forecasts Example

In [36]:
# define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
target = array([[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9],[9,10],[10,11]])

# define generator
n_input = 2
generator = TimeseriesGenerator(series, target, length=n_input, batch_size=1)

# print each sample
for i in range(len(generator)):
    x, y = generator[i]
    print('%s => %s' % (x, y))

[[1. 2.]] => [[3. 4.]]
[[2. 3.]] => [[4. 5.]]
[[3. 4.]] => [[5. 6.]]
[[4. 5.]] => [[6. 7.]]
[[5. 6.]] => [[7. 8.]]
[[6. 7.]] => [[8. 9.]]
[[7. 8.]] => [[ 9. 10.]]
[[8. 9.]] => [[10. 11.]]
